In [ ]:
from fetch import DataFetcher

In [ ]:
#    admin@kasm.local
#    8qz0ykjy3Zs3D

In [ ]:
from datetime import datetime, timezone
import time

symbol = "BTC/USD"
start_time = "2025-09-16T00:00:00Z"  # ISO format for start time
current_time = int(time.time() * 1000)  # Current time in milliseconds
exchange_name = 'coinbase'
time_interval = '1m'
start_time_count = 0
# Create an instance of DataFetcher
fetcher = DataFetcher(symbol, start_time, current_time, exchange_name, start_time_count, time_interval)

# Fetch the data
df = fetcher.fetch_main()


In [ ]:
df

In [ ]:
from segments import RegressionSegments # Import the class
from quantreg import QuantileRegressionLines  # Import the class
import line_properties

In [ ]:
#linear regression segments process
segments = RegressionSegments(df) 

# Step 2: Call the run_all method
regression_segmentation = segments.segmentation_processes()
print(regression_segmentation)
regression_segmentation2 = segments.remove_duplicates_preserve_order(regression_segmentation)

In [ ]:
#quantile regression process
qr = QuantileRegressionLines(df, regression_segmentation2)

df_qr = qr.run_all()
# df_qr goes to plot chart

#Line properties
lines_properties = line_properties.calculate_and_add_properties(df_qr)
lines_coordinates = line_properties.calculate_and_add_coordinates_only(df_qr)
start_time_lower_interval = line_properties.format_datetime_intervals(regression_segmentation2, df, time_interval)

In [ ]:
regression_segmentation2

In [ ]:
regression_segmentation

In [ ]:
segments

In [ ]:
lines_properties

In [ ]:

#code to change the data frame to just works with the original code to plot 
#the issue is that it is expecting a index with the timestamp
import pandas as pd

def change_df_plot(df_filtered):
    # Convert 'date' from ms to datetime and set it as the index
    df_filtered = df_filtered.copy()
    df_filtered.index = pd.to_datetime(df_filtered['date'], unit='ms', utc=True)
    
    # Make sure the index is sorted and unique (good practice for mplfinance)
    df_filtered = df_filtered.sort_index()
    df_filtered = df_filtered[~df_filtered.index.duplicated(keep='last')]
    return df_filtered

df_filtered = change_df_plot(df_qr)

In [ ]:
import pandas as pd        # for handling DataFrames
import matplotlib.pyplot as plt  # optional, but useful if you want extra customization
import mplfinance as mpf   # for candlestick and financial plotting


filename = 'temp_test'

def plot_chart(df_filtered, filename):
    #################PLOT
    #entra la varialbe output de process_quantile_regression que seria de QuantileRegressionLines.run_all

    # Identify the quantile regression columns that passed the correlation filter
    qr_columns = [col for col in df_filtered.columns if 'QR_' in col] # Get all QR columns

    # Create additional plots for the quantile regression lines
    add_plots = [mpf.make_addplot(df_filtered[qr_column], color='blue') for qr_column in qr_columns]

    # Current timestamp to append to the filename
    #filename = f'BTC_USDT_4h_QR_chart.png'

    # Plot the candlestick chart with the quantile regression lines
    mpf.plot(df_filtered, type='candle', addplot=add_plots, style='charles',
             title='BTC/USDT with Quantile Regression Lines', figratio=(35.0, 8.0))#, savefig=filename)


In [ ]:
plot_chart(df_filtered, filename)

In [ ]:
df_qr